In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [42]:
!pip install -U langchain-community langchain-groq

In [7]:
!pip install groq  # Install the groq-client package to use the Groq API

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 3.7 MB/s eta 0:00:00


In [44]:
import getpass
import os

# Set up your Groq API key
os.environ["GROQ_API_KEY"]= getpass.getpass("Enter your Groq API key: ")

from langchain_groq import ChatGroq
model = ChatGroq(model='llama3-8b-8192')

Enter your Groq API key: ··········


In [45]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='According to our conversation, your name is Bob!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 40, 'total_tokens': 51, 'completion_time': 0.009166667, 'prompt_time': 0.001501024, 'queue_time': 0.011703786, 'total_time': 0.010667691}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c4bc8e2-6326-414b-982d-fcea8d02145a-0', usage_metadata={'input_tokens': 40, 'output_tokens': 11, 'total_tokens': 51})

In [46]:
!pip install langgraph


In [47]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [49]:
config = {"configurable": {"thread_id": "abc123"}}

In [50]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! Nice to meet you! Is there something I can help you with today?


In [51]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I already know that! Your name is Bob!


In [52]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous chats. So, I don't actually know your name. Would you like to introduce yourself?


In [53]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [54]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information or remember previous conversations. Each time you interact with me, it's a new conversation, and I don't have any prior knowledge about you. So, I don't know your name. If you'd like to introduce yourself, I'd be happy to chat with you!


In [55]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [56]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [57]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, shiver me timbers! 'Tis a pleasure to make yer acquaintance, Jim me lad! What be bringin' ye to these fair waters?


In [58]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Shiver me spyglass! Ye be askin' yer own name, eh? Well, matey, I be tellin' ye, yer name be Jim! Ye've already told me yerself, savvy?


In [59]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [60]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [61]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola! Soy un asistente amigable. Me alegra conocerte, Bob. ¿En qué puedo ayudarte hoy?


In [62]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Ahah! ¡Eso es fácil! Tu nombre es Bob.


In [63]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [64]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [65]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bob!


In [66]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me what 2 + 2 is.


In [67]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(wait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| smile|!||

In [95]:
import getpass
import os
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages

# Set up your Groq API key
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize the Groq model
model = ChatGroq(model='llama3-8b-8192')

# Define a custom state schema
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

# Define the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Store for session histories
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create a chat history for a given session ID."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap the model with session-specific chat history
with_message_history = RunnableWithMessageHistory(model, get_session_history)

# Define the workflow graph
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    """Generate a response using the wrapped model."""
    prompt = prompt_template.invoke(state)
    response = with_message_history.invoke(prompt)
    return {"messages": [response]}

# Add nodes and edges to the workflow graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory to persist context
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Chatbot function
def dynamic_chatbot():
    print("Welcome to the Dynamic Chatbot with Session Management! Type 'exit' to end the session.\n")

    # Initialize session-specific configuration
    session_id = input("Enter your session ID: ")
    thread_id = session_id  # Use session_id as thread_id for simplicity
    config = {"configurable": {"session_id": session_id, "thread_id": thread_id}}  # Include thread_id here
    language = "English"

    # Initialize the SystemMessage once and keep it fixed
    system_message = SystemMessage(content=f"You are a helpful assistant. Answer all questions to the best of your ability in {language}.")

    while True:
        user_input = str(input("You: "))
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Retrieve session history
        chat_history = get_session_history(session_id)
        chat_history.add_user_message(user_input)

        # Format the messages list correctly, including only relevant messages
        input_messages = [system_message] + chat_history.messages  # Add system message once

        # Ensure the input_messages are in the correct format
        input_messages = [message for message in input_messages if isinstance(message, BaseMessage)]

        # Invoke the chatbot
        output = with_message_history.invoke(
            input_messages,
            config=config,  # Pass the correct config with session_id and thread_id
        )

        # Get the chatbot's response and display it
        response_message = output
        print(f"Chatbot: {response_message.content}")
        chat_history.add_ai_message(response_message.content)

# Run the chatbot
if __name__ == "__main__":
    dynamic_chatbot()


Enter your Groq API key: ··········
Welcome to the Dynamic Chatbot with Session Management! Type 'exit' to end the session.

Enter your session ID: 1
You: what i liketo eat
Chatbot: I'm happy to help you discover new food ideas!

Can you please give me a hint about your food preferences? For example:

* Do you like sweet or savory foods?
* Are you a meat-lover, a vegetarian, or a vegan?
* Do you have a favorite cuisine, such as Italian, Mexican, or Indian?
* Are you looking for something light and healthy or something comforting and indulgent?
* Do you have any dietary restrictions or allergies I should keep in mind?

The more information you provide, the better I can tailor my suggestions to your tastes!
You: show my memory
Chatbot: I apologize, but I don't have any information stored about your food preferences or tastes. Our conversation just started, and I don't have any prior knowledge about your likes or dislikes.

If you'd like to share your favorite foods or cuisines with me, I

KeyboardInterrupt: Interrupted by user

In [ ]:
import getpass
import os
from typing import Sequence
from typing_extensions import Annotated, TypedDict
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages

# Set up your Groq API key
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

# Initialize the Groq model
model = ChatGroq(model='llama3-8b-8192')

# Define a custom state schema
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

# Define the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Store for session histories
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create a chat history for a given session ID."""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Wrap the model with session-specific chat history
with_message_history = RunnableWithMessageHistory(model, get_session_history)

# Define the workflow graph
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    """Generate a response using the wrapped model."""
    prompt = prompt_template.invoke(state)
    response = with_message_history.invoke(prompt)
    return {"messages": [response]}

# Add nodes and edges to the workflow graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory to persist context
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Chatbot function
def dynamic_chatbot():
    print("Welcome to the Dynamic Chatbot with Session Management! Type 'exit' to end the session and 'exitttt' to quit the program.\n")

    while True:
        session_id = input("Enter your session ID (or type 'exitttt' to quit the program): ")

        if session_id.lower() == "exitttt":
            print("Exiting the program...")
            break  # Exit the entire program

        # Initialize session-specific configuration
        thread_id = session_id  # Use session_id as thread_id for simplicity
        config = {"configurable": {"session_id": session_id, "thread_id": thread_id}}  # Include thread_id here
        language = "English"

        # Initialize the SystemMessage once and keep it fixed
        system_message = SystemMessage(content=f"You are a helpful assistant. Answer all questions to the best of your ability in {language}.")

        # Retrieve session history
        chat_history = get_session_history(session_id)

        # Chat loop for the current session
        while True:
            user_input = str(input("You: "))
            if user_input.lower() == "exit":
                print("Ending the current session. Please enter a new session ID.\n")
                break  # Exit current session and ask for a new session ID

            # Add user message to history
            chat_history.add_user_message(user_input)

            # Format the messages list correctly, including only relevant messages
            input_messages = [system_message] + chat_history.messages  # Add system message once

            # Ensure the input_messages are in the correct format
            input_messages = [message for message in input_messages if isinstance(message, BaseMessage)]

            # Invoke the chatbot
            output = with_message_history.invoke(
                input_messages,
                config=config,  # Pass the correct config with session_id and thread_id
            )

            # Get the chatbot's response and display it
            response_message = output
            print(f"Chatbot: {response_message.content}")
            chat_history.add_ai_message(response_message.content)

# Run the chatbot
if __name__ == "__main__":
    dynamic_chatbot()


Welcome to the Dynamic Chatbot with Session Management! Type 'exit' to end the session and 'exitttt' to quit the program.

Chatbot: Nice to meet you, Soundappan! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular?
Chatbot: Your name is Soundappan.
